In [ ]:
import numpy as np
import pandas as pd
import csv
from datetime import datetime
import datetime
import glob
import os
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#Pull in a full month
kick_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join( r'<path to file>\2021\May', 'kickstarter*.csv'))), ignore_index= True)

In [ ]:
#Drop columns I know I won't use for either question (successful or not and how successful)
kick_df.drop(columns={'is_starrable','is_starred','is_backing','friends','converted_pledged_amount','country_displayable_name','currency','currency_symbol','currency_trailing_code','current_currency','fx_rate','location','permissions','photo','profile','slug','source_url','state_changed_at','static_usd_rate','urls','usd_exchange_rate','usd_type'},axis=1,inplace=True,errors='ignore')

In [ ]:
#Drop project that are canceled or live
kick_df = kick_df.drop(kick_df[(kick_df['state'] != 'successful') & (kick_df['state'] != 'failed')].index)

In [ ]:
#Read all dataset for 2016-2020
months = {0:'Jan',1:'Feb',2:'Mar',3:'Apr',4:'May',5:'Jun',6:'Jul',7:'Aug',8:'Sep',9:'Oct',10:'Nov',11:'Dec'}
years = {0:'2016',1:'2017',2:'2018',3:'2019',4:'2020',5:'2021'}

for x in range(5,-1,-1):
    if x == 5:
        y = 3
    else:
        y = 11
    for i in range(y,-1,-1):
        print(months[i]+" "+years[x])
        try:
            temp_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('<path to file>\\'+years[x]+'\\'+months[i], 'kickstarter*.csv'))), ignore_index= True)
            temp_df.drop(columns={'is_starrable','is_starred','is_backing','friends','converted_pledged_amount','country_displayable_name','currency','currency_symbol','currency_trailing_code','current_currency','fx_rate','location','permissions','photo','profile','slug','source_url','state_changed_at','static_usd_rate','urls','usd_exchange_rate','usd_type','unread_messages_count_x', 'unseen_activity_count_x','unread_messages_count_y', 'unseen_activity_count_y', 'unseen_activity_count','unread_messages_count'},axis=1,inplace=True,errors='ignore')
            df3 = pd.merge(kick_df,temp_df,how='outer',indicator='Exist',on='id')
            df3.rename(columns={'blurb_y':'blurb','name_y':'name','pledged_y':'pledged','backers_count_y':'backers_count', 'category_y':'category', 'country_y':'country','created_at_y':'created_at', 'creator_y':'creator', 'deadline_y':'deadline', 'disable_communication_y':'disable_communication','goal_y':'goal', 'launched_at_y':'launched_at', 'spotlight_y':'spotlight', 'staff_pick_y':'staff_pick', 'state_y':'state','usd_pledged_y':'usd_pledged'},inplace=True)
            kick_df = kick_df.merge(df3.loc[df3['Exist'] == 'right_only'],how='outer')
            kick_df.drop(columns={'blurb_x','name_x','pledged_x','backers_count_x', 'category_x', 'country_x', 'created_at_x','creator_x', 'deadline_x', 'disable_communication_x', 'goal_x','launched_at_x', 'spotlight_x', 'staff_pick_x', 'state_x','usd_pledged_x','Exist'},axis=1,inplace=True,errors='ignore')
            kick_df = kick_df.drop(kick_df[(kick_df['state'] != 'successful') & (kick_df['state'] != 'failed')].index)
            kick_df.drop_duplicates(['id'],keep='first',inplace=True)
        except IOError:
            print("File not found")

In [ ]:
#Pull in another month, which will be appended to the first month (only rows that don't match)
temp_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join( r'<path to file>\2016\Jan', 'kickstarter*.csv'))), ignore_index= True)

In [ ]:
#Drop columns I know I won't use for either question (successful or not and how successful)
temp_df.drop(columns={'is_starrable','is_starred','is_backing','friends','converted_pledged_amount','country_displayable_name','currency','currency_symbol','currency_trailing_code','current_currency','fx_rate','location','permissions','photo','profile','slug','source_url','state_changed_at','static_usd_rate','urls','usd_exchange_rate','usd_type'},axis=1,inplace=True,errors='ignore')

In [ ]:
#Only include projects that were either successful or failed
temp_df = temp_df.drop(temp_df[(kick_df['state'] != 'successful') & (temp_df['state'] != 'failed')].index)

In [ ]:
#Final merge on Jan 2016 data
df3 = pd.merge(kick_df,temp_df,how='outer',indicator='Exist',on='id')
df3.rename(columns={'blurb_y':'blurb','name_y':'name','pledged_y':'pledged','backers_count_y':'backers_count', 'category_y':'category', 'country_y':'country','created_at_y':'created_at', 'creator_y':'creator', 'deadline_y':'deadline', 'disable_communication_y':'disable_communication','goal_y':'goal', 'launched_at_y':'launched_at', 'spotlight_y':'spotlight', 'staff_pick_y':'staff_pick', 'state_y':'state','usd_pledged_y':'usd_pledged'},inplace=True)
kick_df = kick_df.merge(df3.loc[df3['Exist'] == 'right_only'],how='outer')
kick_df.drop(columns={'blurb_x','name_x','pledged_x','backers_count_x', 'category_x', 'country_x', 'created_at_x','creator_x', 'deadline_x', 'disable_communication_x', 'goal_x','launched_at_x', 'spotlight_x', 'staff_pick_x', 'state_x','usd_pledged_x','Exist'},axis=1,inplace=True,errors='ignore')
kick_df = kick_df.drop(kick_df[(kick_df['state'] != 'successful') & (kick_df['state'] != 'failed')].index)
kick_df.drop_duplicates(['id'],keep='first',inplace=True)

In [ ]:
#Expand and manipulate creator column
colList_expanded = kick_df['creator'].str.split(',', expand=True)
colList_expanded.columns = ['creator'+str(i) for i in colList_expanded.columns]
kick_df = pd.concat([kick_df,colList_expanded], axis=1)
kick_df['CreatorID'] = ''

In [ ]:
#Add in the CreatorID
for index,row in kick_df.iterrows():  
    print(index)
    for i in range(16):
        string = kick_df.loc[index,'creator'+str(i)]
        if (string[2:4] == 'id'):
            kick_df.loc[index,'CreatorID'] = string[6:]
            break
        elif (string[1:3] == 'id'):
            kick_df.loc[index,'CreatorID'] = string[5:]
            break

In [ ]:
#Drop remaining columns
kick_df.drop(columns={'last_update_published_at','unread_messages_count','unseen_activity_count','unread_messages_count_x','unseen_activity_count_x','unread_messages_count_y','unseen_activity_count_y','last_update_published_at_x','last_update_published_at_y', 'creator', 'creator0', 'creator1', 'creator2','creator3', 'creator4', 'creator5', 'creator6', 'creator7', 'creator8','creator9', 'creator10', 'creator11', 'creator12', 'creator13','creator14', 'creator15',},axis=1,inplace=True,errors='ignore')

In [ ]:
#Data is now clean, moving forward

In [ ]:
#Expand and manipulate category column
#Part 1
kick_df['CatID'] = ''
colList_expanded = kick_df['category'].str.split(',', expand=True)
colList_expanded.columns = ['category'+str(i) for i in colList_expanded.columns]
kick_df = pd.concat([kick_df,colList_expanded], axis=1)
kick_df = kick_df.replace('"','',regex=True)
kick_df = kick_df.replace('{id:','',regex=True)
for index,row in kick_df.iterrows():       
    for element in row:
        if type(element) == str:
            if element.startswith('parent_id:'):
                kick_df.loc[index,'CatID'] = element
kick_df = kick_df.replace('parent_id:','',regex=True)

In [ ]:
#Part 2
kick_df['SubCatID'] = ''
for index,row in kick_df.iterrows():  
    print(index)
    for i in range(9):
        string = kick_df.loc[index,'category'+str(i)]
        if string is not None:
            if (string[0:2] == 'id'):
                kick_df.loc[index,'SubCatID'] = string[3:]
                break
        kick_df.loc[index,'SubCatID'] = kick_df.loc[index,'category0']

In [ ]:
#Drop unnecessary category columns
kick_df.drop(columns={'category','category0','category1','category2','category3','category4','category5','category6','category7','category8'},axis=1,inplace=True,errors='ignore')

In [ ]:
#Drop unnecessary creator and other columns
kick_df.drop(columns={'disable_communication'},axis=1,inplace=True,errors='ignore')

In [ ]:
#Working with dates
#Part 1: Convert date columns from Unix epoch time
kick_df['CreatedDate'] = pd.to_datetime(kick_df['created_at'],unit='s')
kick_df['DeadLineDate'] = pd.to_datetime(kick_df['deadline'],unit='s')
kick_df['LaunchedDate'] = pd.to_datetime(kick_df['launched_at'],unit='s')

In [ ]:
#Part 2: Create columns that show difference between dates and drop original columns
kick_df['DaysDiffCreateLaunch'] = (kick_df['LaunchedDate'] - kick_df['CreatedDate']).dt.days
kick_df['DaysDiffLaunchDeadline'] = (kick_df['DeadLineDate'] - kick_df['LaunchedDate']).dt.days
kick_df.drop(columns={'created_at','deadline','launched_at'},axis=1,inplace=True)

In [ ]:
#Part 3: Create columns that separate out parts of dates
#Note: Day of week start with Monday as zero
#Tough to pull out the week in the month, pursue this if predictions not accurate
#Could also try "day of year", but with a small number of years, this doesn't seem like a good idea
kick_df['CreatedDate_Month'] = kick_df['CreatedDate'].dt.month
kick_df['CreatedDate_DayOfMonth'] = kick_df['CreatedDate'].dt.day
kick_df['CreatedDate_Year'] = kick_df['CreatedDate'].dt.year
kick_df['CreatedDate_DayOfWeek'] = kick_df['CreatedDate'].dt.weekday

kick_df['DeadLineDate_Month'] = kick_df['DeadLineDate'].dt.month
kick_df['DeadLineDate_DayOfMonth'] = kick_df['DeadLineDate'].dt.day
kick_df['DeadLineDate_Year'] = kick_df['DeadLineDate'].dt.year
kick_df['DeadLineDate_DayOfWeek'] = kick_df['DeadLineDate'].dt.weekday

kick_df['LaunchedDate_Month'] = kick_df['LaunchedDate'].dt.month
kick_df['LaunchedDate_DayOfMonth'] = kick_df['LaunchedDate'].dt.day
kick_df['LaunchedDate_Year'] = kick_df['LaunchedDate'].dt.year
kick_df['LaunchedDate_DayOfWeek'] = kick_df['LaunchedDate'].dt.weekday

In [ ]:
kick_df['NumSuccess'] = 0
kick_df['NumFail'] = 0
kick_df['NumPrevSuccess'] = 0
kick_df['NumPrevFail'] = 0

In [ ]:
#Dropping rows with live as the status
kick_df = kick_df[kick_df.state != 'live']

In [ ]:
#New attempt for success/failure
dictCreator = {}
for index,row in kick_df.iterrows():
    CreatorID = row['CreatorID']
    print(CreatorID)
    
    if CreatorID not in dictCreator:    
        dictCreator[CreatorID] = {}
        dictCreator[CreatorID]['NumSuccess'] = 0
        dictCreator[CreatorID]['NumFail'] = 0
        
    kick_df.loc[index,'NumPrevSuccess'] = dictCreator[CreatorID]['NumSuccess']
    kick_df.loc[index,'NumPrevFail'] = dictCreator[CreatorID]['NumFail']
                    
    if row['state'] == 'successful':
        dictCreator[CreatorID]['NumSuccess'] +=1
    elif row['state'] == 'failed':
        dictCreator[CreatorID]['NumFail'] += 1  
        
    kick_df.loc[index,'NumSuccess'] = dictCreator[CreatorID]['NumSuccess']
    kick_df.loc[index,'NumFail'] = dictCreator[CreatorID]['NumFail']

In [ ]:
#Changing to dummy variables
kick_df = pd.get_dummies(kick_df,columns=['spotlight','staff_pick','state'],drop_first=True)

In [ ]:
#Dropping state_failed
kick_df.drop(columns={'state_failed'},axis=1,inplace=True)

In [ ]:
#Final write to CSV
kick_df.to_csv( r'<path to file>\FinalFullDataset.csv',index=False)